In [1]:
import numpy as np
from tqdm import trange
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm

import mowl
mowl.init_jvm('10g')
from mowl.datasets import PathDataset, Dataset
from mowl.base_models import EmbeddingELModel
from mowl.nn import ELEmModule, ELBoxModule, BoxSquaredELModule
from mowl.utils.data import FastTensorDataLoader

from src.utils import *
from src.elmodule import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [3]:
file_name = 'family'
format_ = None
train_graph, test_graph, valid_graph = split_ontology(file_name=file_name, format_=format_, train_ratio=0.8, test_ratio=0.2, add_noise=False)

dataset = PathDataset(ontology_path=f'datasets/bin/{file_name}_train.owl',
                      testing_path=f'datasets/bin/{file_name}_test.owl',
                      validation_path=f'datasets/bin/{file_name}_test.owl')

Triplets found in family.owl: 5017
Train Triplets found: 4013
Test Triplets found: 1004
Valid Triplets found: 0


In [4]:
train_ont = preprocess_ontology_el(dataset.ontology)
test_ont = preprocess_ontology_el(dataset.testing)
valid_ont = preprocess_ontology_el(dataset.validation)

dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)

**Models**

In [5]:
elem = ElModel(dataset, 
               module_name='elem', 
               dim=200, 
               margin=0.1, 
               batch_size=4096*8, 
               test_batch_size=32, 
               epochs=500, 
               learning_rate=0.001,
               device=device)

In [6]:
elem._train()

INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(69 77) 53)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '77'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(56 81)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '81'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(46 ObjectSomeValuesFrom(116 82))
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '82'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(92 20) 94)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '94'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(76 48)
INFO:root:de.tudresden.inf.lat.jce

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.7336, ABox loss: 1.4034
Epoch: 100, Training: EL loss: 3.1272, ABox loss: 0.3677
Epoch: 200, Training: EL loss: 2.4859, ABox loss: 0.0159
Epoch: 300, Training: EL loss: 2.1227, ABox loss: 0.0052
Epoch: 400, Training: EL loss: 1.7633, ABox loss: 0.0075


In [7]:
elbox = ElModel(dataset, 
                module_name='elbox', 
                dim=200, 
                margin=0.1, 
                batch_size=4096*8, 
                test_batch_size=32, 
                epochs=500, 
                learning_rate=0.001,
                device=device)

In [8]:
elbox._train()

INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(69 77) 53)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '77'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(56 81)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '81'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(46 ObjectSomeValuesFrom(116 82))
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '82'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(92 20) 94)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '94'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(76 48)
INFO:root:de.tudresden.inf.lat.jce

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.7442, ABox loss: 0.8360
Epoch: 100, Training: EL loss: 2.8974, ABox loss: 0.2331
Epoch: 200, Training: EL loss: 2.3654, ABox loss: 0.0103
Epoch: 300, Training: EL loss: 1.9039, ABox loss: 0.0009
Epoch: 400, Training: EL loss: 1.7963, ABox loss: 0.0006


In [9]:
box2el = ElModel(dataset, 
                 module_name='box2el', 
                 dim=200, 
                 margin=0.1, 
                 batch_size=4096*8, 
                 test_batch_size=32, 
                 epochs=500, 
                 learning_rate=0.001,
                 device=device)

In [10]:
box2el._train()

INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(69 77) 53)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '77'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(56 81)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '81'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(46 ObjectSomeValuesFrom(116 82))
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '82'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(92 20) 94)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '94'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(76 48)
INFO:root:de.tudresden.inf.lat.jce

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.7239, ABox loss: 0.8363
Epoch: 100, Training: EL loss: 2.8774, ABox loss: 0.2309
Epoch: 200, Training: EL loss: 2.3190, ABox loss: 0.0204
Epoch: 300, Training: EL loss: 2.0002, ABox loss: 0.0038
Epoch: 400, Training: EL loss: 1.7978, ABox loss: 0.0005


**Eval**

In [11]:
elem._eval('subsumption')

Mean Rank: 39.000, Median Rank: 39.000, MRR: 0.025
Hits@1: 0.000, Hits@3: 0.000, Hits@10: 0.000, Hits@100: 1.000


In [12]:
elbox._eval('subsumption')

Mean Rank: 34.000, Median Rank: 34.000, MRR: 0.029
Hits@1: 0.000, Hits@3: 0.000, Hits@10: 0.000, Hits@100: 1.000


In [14]:
box2el._eval('subsumption')

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)